In [ ]:
%load_ext autoreload
%autoreload 2

# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import torch
from pathlib import Path
from solver import Solver
from griddy.griddy_tuna import hit_griddy, SearchMethod
from models.CRNN import CRNNModel
from models.ChromaTransformer import ChromaTransformerModel
from data.data_loader import MirDataProcessor
from utils.model_utils import get_device

In [ ]:
device = get_device()
print(f"Device is {device}")

In [ ]:
# If you have already ran the downloader, change the value of download to False
download = False

# Download and build useable train/test data out of the MIR Billboard dataset
data_processer = MirDataProcessor(download=download, batch_size=64)
if download:
    data_processer.process_billboard_data()

# Create data loeaders for train and test set
train_loader, test_loader, num_classes = data_processer.build_data_loaders(device=device, nrows=2000) # nrows set to shrink dataset for testing

print(f"Number of classes: {num_classes}")

In [ ]:
# NOTE: Do not assume these values are anything but trash, they're just here for testing

SOLVER_PARAMS = {
    Solver : {
        "device": device,
        "batch_size": 64,
        "epochs": 2,
        "early_stop_epochs": 0, # early stop after n epochs without improvement, 0 to disable
        "warmup_epochs": 0, # 0 to disable
        "dtype": "float16",
        "train_dataloader": train_loader, # must be DataLoader object
        "valid_dataloader": test_loader, # must be DataLoader object
        "direction": "minimize" # must specify this, even if not used by solver
    }
}

MODEL_PARAMS = {
    ChromaTransformerModel: {
        "input_dim": [12],
        "seq_length": [100],
        "num_classes": [num_classes],
    }
}

OPTIM_PARAMS = {
    torch.optim.SGD : {
        "lr": [0.001, 0.1, SearchMethod.LOG_UNIFORM],
        "momentum": [0.9, 0.99, SearchMethod.UNIFORM],
        "weight_decay": [0.00001],
    },
    torch.optim.Adam : {
        "lr": [0.03, 0.02, 0.01, 0.1], # this will auto-search as CATEGORICAL
    }
}

SCHED_PARAMS = {
    torch.optim.lr_scheduler.CosineAnnealingWarmRestarts : {
        "T_0": [10],
    },
    torch.optim.lr_scheduler.StepLR : {
        "step_size": [5, 10, 15],
        "gamma" : [0.1, 0.05],
    }
}

CRITERION_PARAMS = {
    torch.nn.CrossEntropyLoss : {}
}

PARAM_SET = {
    "solver": SOLVER_PARAMS,
    "model" : MODEL_PARAMS,
    "optim" : OPTIM_PARAMS,
    "sched" : SCHED_PARAMS,
    "criterion" : CRITERION_PARAMS,
}

In [ ]:
my_study = "chroma_transformer"

output_folder = Path(f"notebooks/{my_study}/griddy/")

solver_reference = hit_griddy(my_study, param_set=PARAM_SET, out_dir=output_folder, n_trials=20, n_jobs=2, prune=False, resume=False)
# NOTE: modest values of n_trials and n_jobs set here for testing, set your values accordingly